In [7]:
from experiments.cubes_2d import get_dataloader, create_model

/mnt/antares_raid/home/jonasb/Projects/wscrl/repo/experiments/cubes_2d.py:50: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(


In [8]:
import logging
from omegaconf import OmegaConf
import torch
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, Subset

torch.set_grad_enabled(False)

logger = logging.getLogger()

EXP_DIR = Path("/mnt/raid/ni/jonasb/wscrl/saved/causal_variables_wrong_top?")

model_path = EXP_DIR / "models" / "model.pt"
# model_path = EXP_DIR / "models" / "model_step_150000.pt"
cfg_path = EXP_DIR / "config.yml"

cfg = OmegaConf.load(cfg_path)

model = create_model(cfg)
model.load_state_dict(torch.load(model_path))
model.eval()

device = torch.device(cfg.training.device)
model.to(device)

CAUSAL_VARIABLES = [
    "pos_x1",
    "pos_y1",
    "pos_x2",
    "pos_y2",
    "pos_x3",
    "pos_y3",
]

CAUSAL_GRAPH = torch.tensor(
    [
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
    ]
)

train_loader = get_dataloader(
    cfg, "train", batchsize=cfg.training.batchsize, shuffle=False, include_noise_encodings=True
)
val_loader = get_dataloader(
    cfg, "val", batchsize=cfg.training.batchsize, shuffle=False, include_noise_encodings=True
)

In [5]:
def read_topological_from_output_log(output_log):
    with open(output_log, "r") as f:
        for line in f:
            if "Topological order" in line:
                return [int(x) for x in line.split("[")[2].split("]")[0].split(", ")]

In [6]:
topological_order = read_topological_from_output_log(EXP_DIR / "output.log")
topological_order

[4, 0, 3, 2, 1, 5]